Setup:
*   Google Colab
*   FastText



# Install packages

In [1]:
! wget https://github.com/facebookresearch/fastText/archive/v0.9.2.zip  > /dev/null
! unzip v0.9.2.zip > /dev/null
%cd fastText-0.9.2
! make > /dev/null
! pip install . > /dev/null

--2023-07-08 14:28:50--  https://github.com/facebookresearch/fastText/archive/v0.9.2.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/facebookresearch/fastText/zip/refs/tags/v0.9.2 [following]
--2023-07-08 14:28:50--  https://codeload.github.com/facebookresearch/fastText/zip/refs/tags/v0.9.2
Resolving codeload.github.com (codeload.github.com)... 140.82.112.10
Connecting to codeload.github.com (codeload.github.com)|140.82.112.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘v0.9.2.zip’

v0.9.2.zip              [  <=>               ]   4.17M  11.5MB/s    in 0.4s    

2023-07-08 14:28:51 (11.5 MB/s) - ‘v0.9.2.zip’ saved [4369852]

/content/fastText-0.9.2


In [2]:
%cd ..

/content


In [3]:
import fasttext
import pandas as pd
import numpy as np
from tqdm import tqdm
import pandas as pd

# Prepare dataset for training

In [8]:
train_dataset = pd.read_csv('train.csv')
train_dataset

,comment,sentiment
0,کس میدونه چه جوری از این ها میشه شکایت کرد لطف...,Negative
1,اف بر شهرداری که درخت را وسط میدان انداخته. طر...,Negative
2,خیلی جای بکری هس حتما یه سر برید👌,Positive
3,آب بسیار کثیف است، متراژ هم کم,Negative
4,افتضاح چون یه شماره تماس نداره خیرسرش,Negative
...,...,...
2538,اصلا کیفیت نداره از سر مجبوری اومدیم اتاق کثیف...,Negative
2539,بسیار عالی بدون سردرد,Positive
2540,برای زیارت و استراحتی کوتاه خوبه در ضمن کتاب ف...,Positive
2541,جای خوبی نیست .یه دونه کافی شاپ امیر شکلات بود...,Negative


In [9]:
train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2543 entries, 0 to 2542
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   comment    2543 non-null   object
 1   sentiment  2543 non-null   object
dtypes: object(2)
memory usage: 39.9+ KB


In [10]:
train_dataset.describe()

,comment,sentiment
count,2543,2543
unique,2543,3
top,کس میدونه چه جوری از این ها میشه شکایت کرد لطف...,Negative
freq,1,1159


In [16]:
# check number of samples per category
print(list(train_dataset['sentiment'] == 'Negative').count(True))
print(list(train_dataset['sentiment'] == 'Positive').count(True))
print(list(train_dataset['sentiment'] == 'Neutral').count(True))

1159
938
446


In [17]:
# final check to find out if there is any null value in dataset
print(train_dataset['comment'].isnull().sum())
print(train_dataset['comment'].isna().sum())

0
0


In [18]:
categories = train_dataset['sentiment'].unique().tolist()
categories

['Negative', 'Positive', 'Neutral']

In [19]:
train_dataset['sentiment_fasttext'] = None

In [174]:
def pre_process(dataset):
    # just a simple pre-processing to make sure
    for index, p in tqdm(enumerate(dataset['comment'])):
        new_comment = p.replace('\n', ' ')
        new_comment = new_comment.strip()
        dataset['comment'][index] = new_comment

    return dataset

train_dataset = pre_process(train_dataset)

2543it [00:00, 9486.11it/s]


In [157]:
# add "__label__" to each of comments, because fasttext need it for its training process
for index, p in tqdm(enumerate(train_dataset['comment'])):
  cat = train_dataset['sentiment'][index]
  label = '__label__'+ str(cat)
  new_sentiment = label + " " + str(p)
  train_dataset['sentiment_fasttext'][index] = new_sentiment

2543it [00:00, 8858.68it/s]


In [158]:
# check if everything is ok!
train_dataset

,comment,sentiment,sentiment_fasttext
0,کس میدونه چه جوری از این ها میشه شکایت کرد لطف...,Negative,__label__Negative کس میدونه چه جوری از این ها ...
1,اف بر شهرداری که درخت را وسط میدان انداخته. طر...,Negative,__label__Negative اف بر شهرداری که درخت را وسط...
2,خیلی جای بکری هس حتما یه سر برید👌,Positive,__label__Positive خیلی جای بکری هس حتما یه سر ...
3,آب بسیار کثیف است، متراژ هم کم,Negative,__label__Negative آب بسیار کثیف است، متراژ هم کم
4,افتضاح چون یه شماره تماس نداره خیرسرش,Negative,__label__Negative افتضاح چون یه شماره تماس ندا...
...,...,...,...
2538,اصلا کیفیت نداره از سر مجبوری اومدیم اتاق کثیف...,Negative,__label__Negative اصلا کیفیت نداره از سر مجبور...
2539,بسیار عالی بدون سردرد,Positive,__label__Positive بسیار عالی بدون سردرد
2540,برای زیارت و استراحتی کوتاه خوبه در ضمن کتاب ف...,Positive,__label__Positive برای زیارت و استراحتی کوتاه ...
2541,جای خوبی نیست .یه دونه کافی شاپ امیر شکلات بود...,Negative,__label__Negative جای خوبی نیست .یه دونه کافی ...


In [159]:
# check if everything is ok!
fasttext.tokenize(train_dataset['sentiment_fasttext'][24])

['__label__Positive',
 'داروخانه',
 'شبانه',
 'روزی،در',
 'موقعیت',
 'مکانی',
 'و',
 'مسیری',
 'که',
 'همیشه',
 'در',
 'دسترس',
 'میباشد']

In [160]:
# split the train.csv to two set: train, test
# the reason behind this is that we do not have actual labels in the provided test.csv
# so, we have to create our own test set based on the train.csv

from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(train_dataset, test_size=0.2, random_state=1, shuffle=True)

In [161]:
print(len(X_train))
print(len(X_test))

2034
509


In [162]:
# export this coloumn as .txt file, because fasttext library need it!
import csv
X_train['sentiment_fasttext'].to_csv('train.txt',
                                       sep=' ',
                                       index=False,
                                       header=False,
                                       quoting = csv.QUOTE_NONE,
                                       quotechar = "",
                                       escapechar = " ")

X_test['sentiment_fasttext'].to_csv('test.txt',
                                       sep=' ',
                                       index=False,
                                       header=False,
                                       quoting = csv.QUOTE_NONE,
                                       quotechar = "",
                                       escapechar = " ")

In [163]:
! head /content/train.txt

__label__Positive  بد  نیست
__label__Negative  یک  کیف  خریدیم.  حلقه  بندش  ضعیف  بود  و  دائم  در  می  اومد.  پرسنل  از  همون  اول  رفتار  مناسبی  نداشتن  و  مشتری  مداری  بلد  نیستن.  برای  تعمیر  هم  کلی  منت  گذاشتن  دست  آخر  هم  درست  نشد.  خلاصه  مواظب  باشید  گول  ظاهر  رو  نخورید
__label__Positive  کاردرست  .  با  وجدان  کاری.دمه  شما  گرم
__label__Negative  داغون  شلوغ  و  کثیف.  بهداشت  صفر  پاتوق  ارازل  و  اوباش
__label__Negative  سف  فلافلی  قدیمی  فردیس  هست  که  کیفیت  غذاهاش  قبلا  عالی  بوده  ولی  در  حال  حاظر  غذاهاش  کیفیت  چندانی  نداره
__label__Neutral  برای  بازدید  پزشک  معتمد  از  ساعت  ۸صبح  الی  ۱۲ظهر  میباشد  غیر  از  روز  های  تعطیل.
__label__Negative  سرویس  بهداشتی  با  اینکه  پولی  بود،  ولی  نظافت  لازم  نداشت.
__label__Positive  بهترین  بیمارستان  مجرب  ترین  کادر  پزشکی
__label__Positive  اینجا  هم  تنوع  داره  هم  کیفیت  ولی  بیشترین  چیزی  که  به  چشمم  اومد  طرز  برخورد  کردنشون  بود  خیلی  خوبن  اینا
__label__Positive  عالی  کارشون  و  منصف  هست

# Train!

In [164]:
# these hyperparameters are selected based on experiment

model = fasttext.train_supervised('train.txt',
                                  wordNgrams=3,
                                  lr=0.6,
                                #   lrUpdateRate=10,
                                  epoch=100,
                                  loss='ns',
                                  seed=123,
                                  label='__label__')

model.test('test.txt')

(509, 0.8408644400785854, 0.8408644400785854)

In [165]:
model.save_model('neshan_camp_model.bin')

In [166]:
# check if it is really ok!
loaded_model = fasttext.load_model('neshan_camp_model.bin')
loaded_model.test('test.txt')

(509, 0.8408644400785854, 0.8408644400785854)

# Predict

In [167]:
# load the provided test.txt
test_dataset = pd.read_csv('test.csv')
test_dataset = pre_process(test_dataset)
test_dataset

449it [00:00, 12049.05it/s]


,comment
0,مکانی زیبا و دیدنی با چشم اندازی زیبا برای علا...
1,روز جمعه داخل نشان زده بود باز است ولی بسته بود
2,سلام متاسفانه با اپراتور متقلبی در این پمپ بنز...
3,محوطه بزرگ و فضای سبز خوبی دارد
4,مزه خوبی نداره شیرینی وکیک هاش
...,...
444,مکانی بسیار زیبا و دلنشین که علاوه بر کاخ موزه...
445,حیف غذا یک خاطر ساز است واشپز کسی که باعشق خا...
446,افتضاح و گرون تر از بقیه
447,مسجد کرمانی یکی از مسجدهای قدیمی شهر تربت جام ...


In [168]:
def single_predict(comment:str, model):
    """
    this function is just for single sample prediction.
    comment: a string
    model: trained model
    """

    result = model.predict(comment)
    label = result[0][0].replace('__label__', '')
    prob = result[1][0]

    return label, prob

In [182]:
def bunch_predict(comments, model):
    """
    this function is able to get a bunch of samples (an array).
    comment: an array of string items
    model: trained model
    """

    labels = []
    probs = []

    for comment in comments:
        result = model.predict(comment)
        labels.append(result[0][0].replace('__label__', ''))
        probs.append(result[1][0])

    return labels, probs

# Example of prediction

In [170]:
single_predict(test_dataset['comment'][1], model)

('Negative', 0.9990700483322144)

In [192]:
comments = test_dataset['comment'].to_list()
labels, probs = bunch_predict(comments, model)

# Prepare new test.csv

In [193]:
new_test = {"comment": [], "sentiment": []}
for label,  comment in zip(labels, comments):
    new_test['comment'].append(comment)
    new_test['sentiment'].append(label)

In [196]:
new_test = pd.DataFrame(new_test)
new_test

,comment,sentiment
0,مکانی زیبا و دیدنی با چشم اندازی زیبا برای علا...,Positive
1,روز جمعه داخل نشان زده بود باز است ولی بسته بود,Negative
2,سلام متاسفانه با اپراتور متقلبی در این پمپ بنز...,Negative
3,محوطه بزرگ و فضای سبز خوبی دارد,Positive
4,مزه خوبی نداره شیرینی وکیک هاش,Negative
...,...,...
444,مکانی بسیار زیبا و دلنشین که علاوه بر کاخ موزه...,Positive
445,حیف غذا یک خاطر ساز است واشپز کسی که باعشق خا...,Negative
446,افتضاح و گرون تر از بقیه,Negative
447,مسجد کرمانی یکی از مسجدهای قدیمی شهر تربت جام ...,Neutral


In [197]:
!mkdir new_test
new_test.to_csv('new_test/test.csv')